# Review the game score data in Metacritic game site
# Part time Data Science course - practice
Try to study the data from the "Metacritic all time games stats" dataset from Kaggle

It contains data of all game listed in metacritic, that are released from 30th April 1995 to 27th December 2020

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Loading the data

In [ ]:
game_score = pd.read_json("/kaggle/input/metacritic-all-time-games-stats/metacritic_games.json")
game_score.head()        

Basic EDA

In [ ]:
game_score.shape

In [ ]:
game_score.info()

In [ ]:
game_score['release_date']=pd.to_datetime(game_score['release_date'])

In [ ]:
game_score.info()

In [ ]:
game_score["user_score"][19]

In [ ]:
game_score.attribute.value_counts()

In [ ]:
game_score.rating.value_counts()

In [ ]:
game_score.players.value_counts()

For "players","rating","attribute", they have the most missing value,
so it is best to remove them from the study.

Also, to compare the score by year,we need to create a year column

In [ ]:
game_score = game_score.drop(columns=["players","rating","attribute","link"])
game_score['year'] = pd.DatetimeIndex(game_score['release_date']).year
game_score.head()

# Metascore vs User_score comparison
Metascore is score by selected critics from online gaming site
User_score are from normal online user comment and review
By comparing the two score we can see if the two data agree with each other

In [ ]:
game_score.user_score.value_counts()

In [ ]:
score=game_score[['metascore','user_score']]
score_v = score[score['user_score']!='tbd']
score_v["user_score"]=pd.to_numeric(score_v["user_score"], downcast="float")
score_v.info()

In [ ]:
g1 = sns.jointplot(x="metascore", y="user_score", data=score_v, kind="hex", color="r", height=15)

Most of the game in the data score around 78 and 7.8 in metascore and user_score. 
(As usually popular game or game from big developer will get review in metacritics)

In [ ]:
g1 = sns.jointplot(x="metascore", y="user_score", data=score_v, kind="reg", color="r", height=15)

But there are some game that the metascore and user_score dont agree

# Metascore of game by year

In [ ]:
game_score.year.value_counts()

In [ ]:
score_year=game_score[['metascore','year']]
score_year.corr()

In [ ]:
f, ax = plt.subplots(figsize=(40, 15))

sns.boxplot(x="year", y="metascore", data=score_year)

Before 1999，the average and mean of the metacritic score is higher. But as there are sigificant less data for game before 1999 (less than 60 for each year), there are bias. (Only famous and good game have record in metacritic, and less popular game have no record)

# Game by developer


In [ ]:
game_dev=game_score[['developer','metascore','year']]
game_dev.sample(10)

In [ ]:
game_dev_c = game_dev[game_dev['developer'].map(game_dev['developer'].value_counts()) > 80]
game_dev_c

In [ ]:
group_dev3 = game_dev_c.groupby('developer')['metascore'].agg(['mean','count']).reset_index().sort_values(by=['mean'], ascending=False)
group_dev3

In [ ]:
group_dev4 = game_dev_c.groupby(['developer','year'])['metascore'].agg(['mean']).reset_index()
group_dev4

In [ ]:
group_dev5 = group_dev4.pivot("developer", "year", "mean")

f, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(group_dev5, annot=True, linewidths=.5, ax=ax)

From the data,
it show Visual Concept and Nintendo is a good developer who continuously make good game
Also, 2007 is a bad year of gaming as most of the developer score lower in that year
